In [34]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [2]:
path = '../Data/Movie_Lens_100k'

In [10]:
train_df = pd.read_csv(os.path.join(path, 'ua.base'), sep = '\t', names = ['user_id', 'movie_id', 'rating', 'timestamp'])
test_df = pd.read_csv(os.path.join(path, 'ua.test'), sep = '\t', names = ['user_id', 'movie_id', 'rating', 'timestamp'])

#### train, test unique 비교

In [18]:
train_user_unique = train_df['user_id'].unique()

In [22]:
set(test_df['user_id'].unique().flatten()) - set(train_df['user_id'].unique().flatten())

set()

In [24]:
test_only_movie = list(set(test_df['movie_id'].unique().flatten()) - set(train_df['movie_id'].unique().flatten()))

In [25]:
test_only_movie

[1653, 1582]

In [28]:
test_df = test_df[~test_df['movie_id'].isin(test_only_movie)]

#### Label Encoder

In [36]:
user_le = LabelEncoder()
movie_le = LabelEncoder()

user_le.fit(train_df['user_id'])
movie_le.fit(train_df['movie_id'])

train_df['user_id'] = user_le.transform(train_df['user_id'])
train_df['movie_id'] = movie_le.transform(train_df['movie_id'])

test_df['user_id'] = user_le.transform(test_df['user_id'])
test_df['movie_id'] = movie_le.transform(test_df['movie_id'])

/tmp/ipykernel_5626/2380308314.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['user_id'] = user_le.transform(test_df['user_id'])
/tmp/ipykernel_5626/2380308314.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['movie_id'] = movie_le.transform(test_df['movie_id'])


#### train.txt, test.txt, train_label.txt, test_label.txt 생성

In [70]:
with open(os.path.join(path, 'train.txt'), 'a') as train_f:
    for user_id, movie_id in zip(train_df['user_id'].unique(), train_df.groupby(['user_id'])['movie_id'].unique()):
        movie_id = movie_id.astype(str)        
        train_f.write(str(user_id) + ' ' + ' '.join(list(movie_id)) + '\n')     

In [72]:
with open(os.path.join(path, 'test.txt'), 'a') as test_f:
    for user_id, movie_id in zip(test_df['user_id'].unique(), test_df.groupby(['user_id'])['movie_id'].unique()):
        movie_id = movie_id.astype(str)        
        test_f.write(str(user_id) + ' ' + ' '.join(list(movie_id)) + '\n')

In [88]:
train_df = train_df.sort_values(by = ['user_id', 'movie_id'])

In [100]:
with open(os.path.join(path, 'train_label.txt'), 'a') as train_f:
    for user_id in train_df['user_id'].unique():
        ratings = train_df[train_df['user_id'] == user_id]['rating'].values.astype(str)        
        train_f.write(str(user_id) + ' ' + ' '.join(list(ratings)) + '\n')

In [101]:
test_df = test_df.sort_values(by = ['user_id', 'movie_id'])

In [102]:
with open(os.path.join(path, 'test_label.txt'), 'a') as test_f:
    for user_id in test_df['user_id'].unique():
        ratings = test_df[test_df['user_id'] == user_id]['rating'].values.astype(str)
        test_f.write(str(user_id) + ' ' + ' '.join(list(ratings)) + '\n')